# Preprocessing
## 2. Extracting Meteorological and GHM variables


The outputs of the GHMs are retrieved from eartH2Observe Water Cycle Integrator (WCI) [data portal](https://wci.earth2observe.eu/portal/?)

These are the steps taken to extract values from the raw NetCDF files.
- We start with caclulating catchment size map following the guide from: https://github.com/edwinkost/catchment_averaging/blob/main/catchment_total_example.md

- We then split the raw_Data into single timesteps, making subsequent analysis much faster
- Lastly we apply catchment normalization using PCRaster ***catchmenttotal*** function.


## Import required  packages

In [4]:
import os
import pcraster as pcr
import numpy as np
from netCDF4 import Dataset
import rioxarray
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import time
from multiprocess import Pool
import tqdm
from os.path import join as pjoin
from osgeo import gdal
import subprocess
import xarray as xr
import rioxarray as rxr

In [2]:
# set your working directory and go there
directory = 'raw_data'
os.chdir(directory)

In [3]:
def check_dir_or_make(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(path)

## 2.1 Catchement averaging

### Transormations cell area and ldd maps

In [4]:
# Define the input and output file paths
in_cell_area = "catchment_area/cellarea30min.nc"
out_cell_area = 'catchment_area/cellarea30min.tif'

in_lddsound = 'catchment_area/lddsound_30min.nc'
out_lddsound = 'catchment_area/lddsound_30min.tif'

# Convert the netCDF file to GeoTIFF using gdal_translate
os.system('gdal_translate {0} {1}'.format(in_cell_area, out_cell_area))
os.system('gdal_translate {0} {1}'.format(in_lddsound, out_lddsound))

0

In [5]:
cmd_cell_area = "gdalwarp -te 4. 44.75 15.25 54.25 -tr 0.5 0.5 -r average " + out_cell_area + " " + "catchment_area/cell_area_EU" + ".tif"
cmd_lddsound = "gdalwarp -te 4. 44.75 15.25 54.25. -tr 0.5 0.5 -r average " + out_lddsound + " " + "catchment_area/lddsound_30min_EU" + ".tif"

os.system(cmd_cell_area)
os.system(cmd_lddsound)

1

In [6]:
# Open the TIFF file
cell_area_EU = gdal.Open("catchment_area/cell_area_EU.tif")

# Get the spatial resolution
geotransform = cell_area_EU.GetGeoTransform()
x_resolution = geotransform[1]
y_resolution = geotransform[5]

# Print the spatial resolution
print("Spatial resolution:")
print("X resolution:", x_resolution)
print("Y resolution:", abs(y_resolution)) 

Spatial resolution:
X resolution: 0.5
Y resolution: 0.5


In [7]:
#Convert the GeoTIFF file to the PCRaster file format using gdal_translate
os.system('gdal_translate -of PCRaster catchment_area/cell_area_EU.tif catchment_area/cellarea30min_EU.map')
os.system('gdal_translate -of PCRaster catchment_area/lddsound_30min_EU.tif catchment_area/lddsound_30min_EU.map')

0

In [9]:
# convert ldd file to a drainage direction type
lddsound_30min_ldd = pcr.lddrepair(pcr.ldd("catchment_area/lddsound_30min_EU.map"))
pcr.report(lddsound_30min_ldd, "catchment_area/lddsound_30min_EU.ldd")

In [10]:
# cell_area and ldd files
cell_area_file = "catchment_area/lddsound_30min_EU.map"
ldd_file      = "catchment_area/lddsound_30min_EU.ldd"

# calculate catchment area

In [11]:
# - set clone, the bounding box of your study area - here, we use ldd 
clone_file     = ldd_file
pcr.setclone(clone_file)

In [12]:
# - read cell_area and ldd files
cell_area = pcr.readmap(cell_area_file)
ldd       = pcr.readmap(ldd_file)

### Calculate the average

In [13]:
## Calculate the average
catchment_area = pcr.catchmenttotal(cell_area, ldd)
# - save catchment_area to a file - note the file output will be under the work_dir
catchment_area_file = "catchment_area/catchment_area.map"
pcr.report(catchment_area, catchment_area_file)

## 2.2 Create single timesteps

In [13]:
def create_single_timestep_maps(var, input_file, model, multi=False):
    check_dir_or_make(model)
    check_dir_or_make(f'{model}/timesteps')
    
    xds = rioxarray.open_rasterio(input_file)
    xds.rio.write_crs("EPSG:4326", inplace=True)  # Set spatial reference to WGS84
    
    if multi:
        variable_names = list(xds.data_vars.keys())[0]
        xds = xds[variable_names]
    
    
    # Print spatial resolution
    print("Spatial resolution:")
    print("X resolution:", xds.rio.resolution()[0])
    print("Y resolution:", xds.rio.resolution()[1])

    
    time = pd.DataFrame(xds[:,0,0].time.to_numpy(), columns=['date'])
    time['date'] = time['date'].astype('str')
    time[['year', 'month', 'left']]= time.date.str.split('-', expand = True)

    for i in range(len(xds[:,0,0])):

        timestep = time.iloc[i,:]
        name = f'{timestep.year}_{timestep.month}'

        
        # Select the single timestep using xarray
        single_timestep = xds.isel(time=i)

        
        single = f'{model}/timesteps/{var}_{name}'
        single_tr = f'{model}/timesteps/{var}_{name}_tr'


        # Save the single timestep as a NetCDF file
        single_timestep.to_netcdf(f'{single}.nc')

        #cmd = f'cdo seltimestep,{i+1} {input_file} {single}.nc'
        #os.system(cmd)
        
        cmd = f'gdal_translate {single}.nc {single}.tif'
        os.system(cmd)
        
        
        # Set the spatial resolution to 0.5x0.5 
        # Convert GeoTIFF to the desired resolution using gdal_translate
        cmd = f"gdalwarp -te 4. 44.75 15.5 54.25 -tr 0.5 0.5 -r average {single}.tif {single_tr}.tif"
        os.system(cmd)


        cmd = f"gdal_translate -of PCRaster {single_tr}.tif {single}.map"
        os.system(cmd)
        
         # Clean up temporary files
        filelist = glob.glob(f'{model}/timesteps/*.xml')
        for file in filelist:
            os.remove(file)

        filelist = glob.glob(f'{model}/timesteps/*.tif')
        for file in filelist:
            os.remove(file)

        filelist = glob.glob(f'{model}/timesteps/*.nc')
        for file in filelist:
            os.remove(file)

In [55]:
## for one file
input_file     = "meteo/meteo_rain.nc"
create_single_timestep_maps('meteo_rain', input_file,'meteo', multi=False)

Spatial resolution:
X resolution: 0.25
Y resolution: -0.25


In [56]:
input_file     = "meteo/meteo_tair.nc"
create_single_timestep_maps('meteo_tair', input_file,'meteo')

Spatial resolution:
X resolution: 0.25
Y resolution: -0.25


In [58]:
models = ["lis","pcr", "wg3"]

for model in models:
    for file in os.listdir(model):
        if file.endswith(".nc"):
            
            var = file.split(".")[0]
            
            input_file = pjoin(model, file)
            create_single_timestep_maps(var, input_file, model, multi=True)
    print("Finished for variables set", model)

Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Finished for variables set lis
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Finished for variables set pcr
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Spatial resolution:
X resolution: 0.5
Y resolution: -0.5
Finished for variables set wg3


In [59]:
# - make sure that the file has the same map attributes as the ldd file
#cmd = f"mapattr -c catchment_area/lddsound_30min_EU.ldd {input_file}.map"
#os.system(cmd)

## 2.3 Calculate upstream averaged values for catchment areas

In [14]:
%%time
def upstream_maps(model, var):
    check_dir_or_make(f'{model}/upstream')

    
    input_files = glob.glob(f'{model}/timesteps/*.map')
    cmd = f'rm {model}/upstream/*'
    os.system(cmd)

    time.sleep(5)
    
    for i in range(len(input_files)):
        name = input_files[i].split('/')[-1]
        cell_value = pcr.readmap(f"{input_files[i]}")
        
        if "dis" in input_files[i]:
            upstream_average_value = cell_value
        
        else:
            
            # calculate upstream/catchment average values
            upstream_average_value = pcr.catchmenttotal(cell_value * cell_area, ldd) / catchment_area
        pcr.report(upstream_average_value, f"{model}/upstream/upstream_{name}")
        
        file_map = f'{model}/upstream/upstream_{name}'
        file_tiff = file_map.replace('.map', '.tif')
        file_nc = file_map.replace('.map', '.nc')
        

        cmd_tif = f'gdal_translate -of GTiff {file_map} {file_tiff}'
        os.system(cmd_tif)
        
        data_array = rxr.open_rasterio(file_tiff)
        data_array.to_netcdf(file_nc)

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
upstream_maps('meteo', 'meteo_rain')

In [16]:
upstream_maps('meteo', 'meteo_tair')

In [17]:
models = ["pcr", "wg3", "lis"]

In [18]:
for model in models:
    for file in os.listdir(model):
        if file.endswith(".nc"):
            
            var = file.split(".")[0]
            upstream_maps(model, var)
    print("Finished for variables set", model)

Finished for variables set pcr
Finished for variables set wg3
Finished for variables set lis
